# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Exploring data

In [2]:
data_set_link = 'https://archive-beta.ics.uci.edu/dataset/352/online+retail'
retail = pd.read_excel('Online Retail.xlsx')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
retail.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [4]:
retail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
retail.shape

(541909, 8)

In [6]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


# Data Preparation

In [9]:
retail.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [10]:
retail.dropna(inplace = True)

In [12]:
retail.shape

(406829, 8)

In [14]:
retail.groupby('Description').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending = False).head(10)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409
POPCORN HOLDER,30504
RABBIT NIGHT LIGHT,27094
MINI PAINT SET VINTAGE,25880
PACK OF 12 LONDON TISSUES,25321


In [15]:
retail['InvoiceNo'].str.contains('C').count()

8905

InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 

In [18]:
retail = retail[~retail['InvoiceNo'].str.contains('C', na = False)]
retail.shape

(397924, 8)

In [19]:
retail['revenue'] = retail['Quantity'] * retail['UnitPrice']
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


# RFM Analysis

In [21]:
import datetime as dt

In [31]:
retail['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [33]:
today_date = dt.datetime(2011,12,10)

In [35]:
rfm = retail.groupby('CustomerID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                       "InvoiceNo": lambda num: num.nunique(),
                                       "revenue": 'sum'})
rfm.head()

,InvoiceDate,InvoiceNo,revenue
CustomerID,,,
12346.0,324,1,77183.60
12347.0,1,7,4310.00
12348.0,74,4,1797.24
12349.0,17,1,1757.55
12350.0,309,1,334.40


In [36]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,324,1,77183.60
12347.0,1,7,4310.00
12348.0,74,4,1797.24
12349.0,17,1,1757.55
12350.0,309,1,334.40


In [39]:
rfm = rfm[rfm['Monetary'] > 0]

In [40]:
rfm.describe()

,Recency,Frequency,Monetary
count,4338.000000,4338.000000,4338.000000
mean,91.059474,4.272706,2054.266460
std,100.012264,7.706221,8989.230441
min,-1.000000,1.000000,3.750000
25%,16.000000,1.000000,307.415000
50%,49.000000,2.000000,674.485000
75%,140.750000,5.000000,1661.740000
max,372.000000,210.000000,280206.020000


In [48]:
rfm['recency_score'] = pd.qcut(rfm['Recency'], 5, labels = [100,80,60,40,20])
rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method= 'first'), 5, labels = [20,40,60,80,100])
rfm['monetary_score'] = pd.qcut(rfm['Monetary'], 5, labels = [20,40,60,80,100])

C:\Users\WhamClinics 2\AppData\Local\Temp\ipykernel_6664\2737542557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['recency_score'] = pd.qcut(rfm['Recency'], 5, labels = [100,80,60,40,20])
C:\Users\WhamClinics 2\AppData\Local\Temp\ipykernel_6664\2737542557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method= 'first'), 5, labels = [20,40,60,80,100])
C:\Users\WhamClinics 2\AppData\Local\Temp\ipykernel_6664\2737542557.py:3: SettingWithCopyW

In [51]:
rfm['RFM_SCORE'] = (rfm['recency_score'].astype('int64') * 0.15 + \
                    rfm['frequency_score'].astype('int64') * 0.28 + rfm['monetary_score'].astype('int64') * 0.57)

C:\Users\WhamClinics 2\AppData\Local\Temp\ipykernel_6664\624062082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['RFM_SCORE'] = (rfm['recency_score'].astype('int64') * 0.15 + \


,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
CustomerID,,,,,,,
12346.0,324,1,77183.60,20,20,100,65.6
12347.0,1,7,4310.00,100,100,100,100.0
12348.0,74,4,1797.24,40,80,80,74.0
12349.0,17,1,1757.55,80,20,80,63.2
12350.0,309,1,334.40,20,20,40,31.4
12352.0,35,8,2506.04,60,100,100,94.0
12353.0,203,1,89.00,20,20,20,20.0
12354.0,231,1,1079.40,20,20,80,54.2
12355.0,213,1,459.40,20,20,40,31.4


In [59]:
rfm['Segment'] = pd.cut(rfm['RFM_SCORE'], bins=[20, 40, 60, 80, 100], labels=['Lowest', 'Low','Medium', 'Premium'])
rfm.head(10)

C:\Users\WhamClinics 2\AppData\Local\Temp\ipykernel_6664\2204887212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['Segment'] = pd.cut(rfm['RFM_SCORE'], bins=[20, 40, 60, 80, 100], labels=['Lowest', 'Low','Medium', 'Premium'])


,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,Segment
CustomerID,,,,,,,,
12346.0,324,1,77183.60,20,20,100,65.6,Medium
12347.0,1,7,4310.00,100,100,100,100.0,Premium
12348.0,74,4,1797.24,40,80,80,74.0,Medium
12349.0,17,1,1757.55,80,20,80,63.2,Medium
12350.0,309,1,334.40,20,20,40,31.4,Lowest
12352.0,35,8,2506.04,60,100,100,94.0,Premium
12353.0,203,1,89.00,20,20,20,20.0,NaN
12354.0,231,1,1079.40,20,20,80,54.2,Low
12355.0,213,1,459.40,20,20,40,31.4,Lowest
